# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [15]:
import pandas as pd
import numpy as np

## imports for Python, Pandas

In [16]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [17]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [18]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

name  population
0        Dade       12345
1     Broward       40000
2  Palm Beach       60000
3      Summit        1234
4    Cuyahoga        1337

In [19]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

name  population    state shortname info.governor
0        Dade       12345  Florida        FL    Rick Scott
1     Broward       40000  Florida        FL    Rick Scott
2  Palm Beach       60000  Florida        FL    Rick Scott
3      Summit        1234     Ohio        OH   John Kasich
4    Cuyahoga        1337     Ohio        OH   John Kasich

****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [20]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [21]:
# load json as string
data = json.load((open('data/world_bank_projects.json')))

# converting the string to pandas dataframe
worldbank= pd.read_json('data/world_bank_projects.json')
worldbank.head(2)


_id  approvalfy board_approval_month  \
0  {'$oid': '52b213b38594d8a2be17c780'}        1999             November   
1  {'$oid': '52b213b38594d8a2be17c781'}        2015             November   

      boardapprovaldate                                 borrower  \
0  2013-11-12T00:00:00Z  FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA   
1  2013-11-04T00:00:00Z                    GOVERNMENT OF TUNISIA   

            closingdate                              country_namecode  \
0  2018-07-07T00:00:00Z  Federal Democratic Republic of Ethiopia!$!ET   
1                   NaN                      Republic of Tunisia!$!TN   

  countrycode                              countryname countryshortname  \
0          ET  Federal Democratic Republic of Ethiopia         Ethiopia   
1          TN                      Republic of Tunisia          Tunisia   

                         ...                           sectorcode source  \
0                        ...                          ET,BS,ES,EP   IBRD   
1                        ...                                BZ,BS   IBRD   

   status  supplementprojectflg  \
0  Active                     N   
1  Active                     N   

                                              theme1  \
0      {'Percent': 100, 'Name': 'Education for all'}   
1  {'Percent': 30, 'Name': 'Other economic manage...   

                                      theme_namecode themecode   totalamt  \
0      [{'code': '65', 'name': 'Education for all'}]        65  130000000   
1  [{'code': '24', 'name': 'Other economic manage...     54,24          0   

  totalcommamt                                                url  
0    130000000  http://www.worldbank.org/projects/P129828/ethi...  
1      4700000  http://www.worldbank.org/projects/P144674?lang=en  

[2 rows x 50 columns]

In [22]:
# information on the columns'name/data type and total number of the attributes

worldbank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 50 columns):
_id                         500 non-null object
approvalfy                  500 non-null int64
board_approval_month        500 non-null object
boardapprovaldate           500 non-null object
borrower                    485 non-null object
closingdate                 370 non-null object
country_namecode            500 non-null object
countrycode                 500 non-null object
countryname                 500 non-null object
countryshortname            500 non-null object
docty                       446 non-null object
envassesmentcategorycode    430 non-null object
grantamt                    500 non-null int64
ibrdcommamt                 500 non-null int64
id                          500 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
lendinginstr                495 non-null object
lendinginstrtype            495 non

In [23]:
# 1. Find the 10 countries with most projects

#subsetting the total number of projects as per countries

p=worldbank.groupby('countryshortname').size().sort_values(ascending=False)
p.head(10)

countryshortname
Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Nepal                 12
Bangladesh            12
Morocco               12
Mozambique            11
Africa                11
dtype: int64

In [24]:
#2.Find the top 10 major project themes (using column 'mjtheme_namecode')
project = json_normalize(data, 'mjtheme_namecode', ['countryname'])
project.code = project.code.astype(int)
project_theme = project.groupby('name').size().sort_values(ascending=False)
project_theme

name
Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Rule of law                                      12
dtype: int64

In [25]:
# 3. In 2. above you will notice that some entries have only the code and the name is missing. 
# Create a dataframe with the missing names filled in.

project = project.sort_values('code')
project.name = project.name.replace('', np.NAN)
project = project.fillna(method='bfill')
project.head(50)


code                      name                               countryname
1114     1       Economic management                      Republic of Honduras
647      1       Economic management          Lao People's Democratic Republic
1260     1       Economic management               Hashemite Kingdom of Jordan
648      1       Economic management          Lao People's Democratic Republic
1437     1       Economic management                     Republic of Guatemala
1257     1       Economic management                      Republic of Maldives
454      1       Economic management  Democratic Republic of Sao Tome and Prin
453      1       Economic management                        Kingdom of Lesotho
1057     1       Economic management                    Republic of Azerbaijan
1024     1       Economic management               United Republic of Tanzania
175      1       Economic management              Islamic State of Afghanistan
249      1       Economic management                       Republic of Armenia
1056     1       Economic management                    Republic of Azerbaijan
1218     1       Economic management                          Kingdom of Tonga
1212     1       Economic management                         Republic of Kenya
803      1       Economic management                        Republic of Panama
1206     1       Economic management                           Pacific Islands
458      1       Economic management  Democratic Republic of Sao Tome and Prin
204      1       Economic management                           Kyrgyz Republic
1235     1       Economic management              Oriental Republic of Uruguay
1045     1       Economic management    Macedonia, former Yugoslav Republic of
2        1       Economic management                       Republic of Tunisia
220      1       Economic management                       Republic of Tunisia
222      1       Economic management               Hashemite Kingdom of Jordan
223      1       Economic management                    Republic of Mozambique
88       1       Economic management                    Republic of Seychelles
212      1       Economic management                       Republic of Liberia
784      1       Economic management               United Republic of Tanzania
205      1       Economic management                           Kyrgyz Republic
900      1       Economic management                         Republic of Yemen
1010     1       Economic management          Republic of the Union of Myanmar
841      1       Economic management             Socialist Republic of Vietnam
1230     1       Economic management                      Republic of Colombia
357      1       Economic management                        Republic of Poland
1078     1       Economic management                                   Ukraine
1229     1       Economic management                      Republic of Colombia
363      1       Economic management                 Republic of Cote d'Ivoire
497      1       Economic management                        Republic of Malawi
516      2  Public sector governance                        West Bank and Gaza
507      2  Public sector governance                     Republic of Indonesia
431      2  Public sector governance                       Antigua and Barbuda
202      2  Public sector governance                           Kyrgyz Republic
502      2  Public sector governance                         Republic of Niger
517      2  Public sector governance                        West Bank and Gaza
501      2  Public sector governance          Lao People's Democratic Republic
500      2  Public sector governance          Lao People's Democratic Republic
496      2  Public sector governance                        Republic of Malawi
211      2  Public sector governance                       Republic of Liberia
491      2  Public sector governance                        Kingdom of Morocco
1173     2  Public sector governance                    